In [1]:
import pathlib
from src.dataset import Dataset, BAD_MOOD

DATA_DIR = pathlib.Path('data')
path = next(DATA_DIR.glob('*.csv'))
print('using file', path.name)

df = Dataset(path)
df.stats()

using file daylio_export_2024_12_05 (2).csv
Dataset(1800 entries; last [5 hours 51 minutes 28 seconds ago]; mood: 3.839 ± 0.500)


Mood: 3.839 ± 0.500
Note length: 47.988 ± 56.116 symbols
Entries frequency: 5.737 entries per day (once every 4 hours 11 minutes)

# API

### `.head`

Use the `.head` method to look at the latest entries of the dataset

In [2]:
df.head()

Dataset(1792 entries; last [14 hours 53 minutes 25 seconds ago]; mood: 3.840 ± 0.500)
[03.12.2024 00:09] 4.0 amused, home, reading, relaxed, satisfied
[02.12.2024 22:04] 3.5 Leha, bored, coding, friends, home, relaxed
[02.12.2024 17:18] 4.0 excited, sad, work
[02.12.2024 14:54] 4.0 coding, productive, unsure, work, worked
[02.12.2024 00:06] 4.0 home, reading, satisfied
...


In [3]:
df.head(2)

Dataset(1792 entries; last [14 hours 53 minutes 27 seconds ago]; mood: 3.840 ± 0.500)
[03.12.2024 00:09] 4.0 amused, home, reading, relaxed, satisfied
[02.12.2024 22:04] 3.5 Leha, bored, coding, friends, home, relaxed
...


In [ ]:
# prints all entries
df.head(-1)

### `.sub`

Use the `.sub` method to filter entries and get a subset of the original dataset

- by included activities ('or' operator: all the entries which have at least one of the listed activities)

In [4]:
cycling_or_city_df = df.sub(include={'cycling', 'city'})
cycling_or_city_df.head()

Dataset(100 entries; last [1 day 1 hour 40 minutes 42 seconds ago]; mood: 4.135 ± 0.541)
[27.07.2024 21:39] 4.5 city, relaxed, satisfied, tired
[23.07.2024 17:35] 4.0 cycling, relaxed, study, unsure
[21.07.2024 19:53] 4.0 cycling, relaxed, unsure
[20.07.2024 18:32] 4.5 city, cycling, relaxed, satisfied
[17.07.2024 20:28] 4.0 city, excited, satisfied, social
...


- by excluded activities (entries which don't have any of the listed activities)

_for both `include` and `exclude` the argument can be either `str` or `set[str]`_

In [5]:
without_friends = df.sub(exclude='friends')
without_friends.head()

Dataset(1213 entries; last [1 hour 33 minutes 43 seconds ago]; mood: 3.799 ± 0.515)
[28.07.2024 21:46] 3.5 home, study, unsure
[28.07.2024 15:57] 4.0 Leha, factorio, home, relaxed, satisfied
[28.07.2024 13:01] 4.0 excited, home, relaxed, satisfied, study
[27.07.2024 21:39] 4.5 city, relaxed, satisfied, tired
[27.07.2024 16:42] 4.5 excited, happy, home, piano, relaxed, satisfied
...


- by mood values (can be either a value or a Container of values)

In [6]:
bad_mood = df.sub(mood=BAD_MOOD)
bad_mood

Dataset(37 entries; last [7 days 17 hours 45 minutes 44 seconds ago]; mood: 2.270 ± 0.253)

- by included activities ('and' operator: all entries which have all the listed activities)

_Note: this is achieved by subscribing the dataset twice; since .sub method returns a new dataset, it can be done in one line._

In [7]:
cycling_and_swimming = df.sub(include='cycling').sub(include='swimming')
cycling_and_swimming.head()
print(cycling_and_swimming.activities().most_common(7))

Dataset(11 entries; last [30 days 51 minutes 45 seconds ago]; mood: 4.773 ± 0.518)
[28.06.2024 22:28] 4.0 cycling, relaxed, satisfied, swimming, tired
[13.10.2023 20:10] 5.0 cycling, swimming, tired
[16.09.2023 20:58] 5.0 cycling, relaxed, satisfied, swimming, tired
[04.09.2023 20:44] 5.0 Damir, cycling, relaxed, satisfied, swimming, tired
[22.08.2023 20:27] 5.0 cycling, friends, satisfied, swimming, tired
...
[('cycling', 11), ('tired', 11), ('swimming', 11), ('satisfied', 8), ('relaxed', 7), ('friends', 6), ('Damir', 5)]


In [8]:
df.sub(include='cycling', exclude='swimming')

Dataset(38 entries; last [5 days 5 hours 44 minutes 46 seconds ago]; mood: 4.145 ± 0.544)

- by a subtring (or substrings) in the `note`

In [9]:
df_ktane = df.sub(note_contains='ktane')
df_ktane.head(-1)

Dataset(6 entries; last [2 months 20 days 11 hours 44 minutes 47 seconds ago]; mood: 3.833 ± 0.258)
[08.05.2024 15:35] 3.5 Leha, annoyed, excited, friends, gaming, unsure
[08.03.2024 17:26] 4.0 Leha, amused, annoyed, friends, gaming, happy, home, satisfied
[22.11.2023 21:41] 4.0 Leha, Martina, amused, gaming, home, movies & series, satisfied
[07.07.2023 20:00] 3.5 Leha, coding, dota, friends, gaming, shopping, study, unsure, walking
[03.07.2023 20:05] 4.0 Leha, coding, excited, friends, gaming, happy, home, stressed, tired
[01.07.2023 20:00] 4.0 Leha, Martina, excited, friends, gaming, happy, home, movies & series, relaxed


- by a predicate function (which takes an entry as an argument and returns a boolean)

In [ ]:
from dataset import Entry


def pred(entry: Entry) -> bool:
    """It is sunday, the mood > 4, the note is not empty"""
    return entry.full_date.weekday() == 6 and entry.mood > 4 and entry.note != ""


df_sunday_good_mood_has_note = df.sub(predicate=pred)
df_sunday_good_mood_has_note

Dataset(31 entries; last [1 month 11 days 15 hours 12 minutes 48 seconds ago]; mood: 4.677 ± 0.243)

### `.mood` and `.std`

Use the `.mood` method to get the average mood of all the entries in the dataset and the `.std` method to get the standard deviation of the mood values.

These values are also shown when calling `.head()`.

In [11]:
print(f"{cycling_or_city_df.mood():.3f} \u00B1 {cycling_or_city_df.std():.3f}")

4.135 ± 0.541


or just use `.mood_std`:

In [12]:
cycling_or_city_df.mood_std()

4.135 ± 0.541

### `.activities`

Use the `.activities` method to get a Counter object of all activities in the dataset

In [13]:
cnt = df.activities()
print(f'most common: {cnt.most_common(3)}')
print(f'least common: {cnt.most_common()[-1:-6:-1]}')

most common: [('home', 800), ('relaxed', 525), ('satisfied', 490)]
least common: [('cinema', 3), ('reading', 3), ('photography', 3), ('guitar', 4), ('Dad', 4)]


### `.people`

Use the `.people` method to get a Counter object of all people in the dataset

In [14]:
df.people()

Counter({'Leha': 50,
         'Mom': 21,
         'Martina': 42,
         'Azat': 61,
         'Damir': 37,
         'Dad': 4,
         'Lion': 42,
         'Lika': 13,
         'Lauren': 254})

### `.get_datetimes`

Use the `.get_datetimes` method to get the list of all points in time when an entry was created. The values are sorted in descending order by the date of creation.

In [16]:
df.get_datetimes()[:5]

[datetime.datetime(2024, 7, 28, 21, 46),
 datetime.datetime(2024, 7, 28, 15, 57),
 datetime.datetime(2024, 7, 28, 13, 1),
 datetime.datetime(2024, 7, 27, 21, 39),
 datetime.datetime(2024, 7, 27, 16, 42)]

### `.group_by`
Use the `.group_by` method to get a dictionary mapping groups to the list of entries in that group.

The groups are one of `['day', 'month']` and the entries are sorted in ascending order.

Note: this method (like many others) uses lru_cache and since the Dataset is weakly-immutable, it is safe to use it.

In [17]:
groups = df.group_by('day')
for day, entries in groups.items():
    print(f'{day}: {len(entries)}')
    break

2023-07-01: 1


In [18]:
groups = df.group_by('month')
for day, entries in groups.items():
    print(f'{day}: {len(entries)}')
    break

2023-07-01: 95


### `.stats`

Use the `.stats` method to get a custom StatsResult object which contains the following information:

In [19]:
help(df.stats)

Help on method stats in module dataset:

stats() -> utils.StatsResult method of dataset.Dataset instance
    Returns the following statistics:
        - mood (avg ± std)
        - note length [num symbols] (avg ± std)
        - entries frequency [entries per day] (median)
    as a StatsResult object.



In [20]:
df.sub(include='home').stats()

Mood: 3.774 ± 0.477
Note length: 36.830 ± 54.716 symbols
Entries frequency: 4.091 entries per day (once every 5 hours 52 minutes)

### `for _ in df` (`.__iter__`)
`Dataset` defines `__iter__` method, so it can be used in `for` loops. The entries are sorted in descending order by the date of creation.

In [21]:
for entry in df:
    print(entry)
    break
# or
df_iter = iter(df)
print(next(df_iter), next(df_iter), sep='; ')

[28.07.2024 21:46] 3.5 home, study, unsure
[28.07.2024 21:46] 3.5 home, study, unsure; [28.07.2024 15:57] 4.0 Leha, factorio, home, relaxed, satisfied


### `df[...]` (`.__getitem__`)
`Dataset` defines `__getitem__` method where the argument is 
- a date in a string format (e.g. `'01.01.2024'`), returns a new Dataset with all entries on that date;
- a slice object (e.g. `slice('01.01.2024', '01.01.2025')`), returns a new Dataset with all entries which were created between the two dates (the "stop" date is not included);

In [22]:
july22 = df['22.07.2023']

In [23]:
july22

Dataset(6 entries; last [1 year 7 days 1 hour 3 minutes 12 seconds ago]; mood: 3.583 ± 0.376)

- by a particular date period (by using slices)

In [24]:
ny_eve = df['29.12.2023':'03.01.2024']
ny_eve

Dataset(16 entries; last [6 months 25 days 11 hours 25 minutes 16 seconds ago]; mood: 3.969 ± 0.499)

In [25]:
end_of_november_2023 = df['29.11.2023':'01.12.2023']
end_of_november_2023.head()

Dataset(5 entries; last [7 months 28 days 1 hour 43 minutes 17 seconds ago]; mood: 3.700 ± 0.274)
[30.11.2023 23:37] 3.5 Lauren, city, date, tired
[30.11.2023 17:06] 4.0 coding, excited, home, satisfied
[30.11.2023 11:07] 3.5 Leha, coding, home, unsure
[29.11.2023 21:20] 3.5 Azat, Leha, amused, gaming, social
[29.11.2023 16:42] 4.0 coding, excited, happy, home, satisfied


In [26]:
before_aug_2023 = df[:'01.08.2023']
before_aug_2023.head()

Dataset(95 entries; last [11 months 28 days 14 hours 26 minutes 18 seconds ago]; mood: 3.589 ± 0.688)
[31.07.2023 18:54] 3.5 cleaning, relaxed
[31.07.2023 17:22] 3.0 home, study, tired, unsure
[31.07.2023 16:45] 4.0 cooking, home, relaxed, satisfied, study
[31.07.2023 12:00] 3.5 bored, home, procrastinating, relaxed, unsure
[30.07.2023 23:26] 4.0 Lauren, amused, friends, grateful, happy, home, laundry
...


### `df @ datetime-like` (or `.at`)
Returns a single entry created at a particular datetime or `None` if there is no entry at that datetime.

Input: a datetime in a string format (e.g. `'01.01.2024 12:00'`) or a `datetime.datetime` object.

In [27]:
import datetime

entry1 = df @ '22.07.2023 17:09' # or df.at(...)
entry2 = df @ datetime.datetime(2023, 7, 22, 17, 9) # or df.at(...)

assert entry1 == entry2
print(entry1)

[22.07.2023 17:09] 3.5 cooking, home, relaxed, study


# Analysis examples

## Mood analysis

In [28]:
activity = 'cycling'
mood_with_without = df.mood_with_without(activity)
print(repr(mood_with_without))
print(mood_with_without.calc_change())

MoodWithWithout(with_=4.286 ± 0.595, without=3.799 ± 0.509)
0.12798012523426947


In [29]:
print(activity, mood_with_without, sep='\n')

cycling
with:    4.286 ± 0.595
without: 3.799 ± 0.509
change:  12.80%


## Complete analysis

In [30]:
complete_analysis = df.complete_analysis()

In [ ]:
print(f'analysed {len(complete_analysis)} actvities')
for _name, _mood_with_without, _num_occ in complete_analysis[:8] + complete_analysis[-8:]:
    print(f'[{_name:^15}]: {_mood_with_without.calc_change():.1%} (with: {_mood_with_without.with_}, without: {_mood_with_without.without}); occured {_num_occ} times')

## Tags

In [2]:
all_tags = df.build_tags()

In [ ]:
for booktag in all_tags['книга']:
    print(booktag)

### Books Timeline

In [ ]:
from IPython.core.display import HTML
from IPython.display import display

from src.books_timeline import get_timeline_html

book_tags = df.get_book_tags()

display(HTML(get_timeline_html(book_tags)))


## Mood scatter plot (per day/month)

In [5]:
df.mood_plot('day')

In [33]:
df.mood_plot('week')

In [34]:
df.mood_plot('month')

## Books Read

In [ ]:
df.books_read_plot()

## Mood bar plot by hour/day/month

In [35]:
df.by_time_bar_plot('hour')

In [36]:
df.by_time_bar_plot('day')

In [37]:
df.by_time_bar_plot('weekday')

In [38]:
df.by_time_bar_plot('month')

## Entries times differences

In [39]:
df.sub(include='home').entries_differences()

## Monthly activity effect on mood

In [42]:
df.mood_change_activity('study')

No 'study' in December 2023


## Other features

In [43]:
df.note_length_plot()

In [2]:
df.generate_activity_correlation_matrix()

# Monthly Report Templates

In [3]:
import datetime


def generate_report_template(month: int, year: int):
    month_init = month
    _from = f'01.{month:02d}.{year}'
    month = month + 1 if month < 12 else 1
    year = year + 1 if month == 1 else year
    _to = f'01.{month:02d}.{year}'
    df_month = df[_from:_to]
    df_month_groups = df_month.group_by('day')

    EVENTS_COMMENT = f'total entries: {len(df_month)}\n\n'
    for day, entries in df_month_groups.items():
        EVENTS_COMMENT += f' -- {day:%d.%m.%Y, %a} --\n'
        for e in entries:
            EVENTS_COMMENT += f'@{e.full_date.time():%H:%M}: {e.mood} {", ".join(e.activities)}\n  {e.note}\n'
        EVENTS_COMMENT += '\n'
    
    month_word = datetime.date(1900, month_init, 1).strftime('%B')
    with open(f'{year}-{month_init:02d}.md', 'w', encoding='utf-8') as f:
        f.write(
            f'# {month_word} {year}\n\n'
            f'<!---\n{EVENTS_COMMENT}--->'
        )

In [4]:
generate_report_template(7, 2024)

# Other